In [2]:
import json
import numpy as np
import os
import pandas as pd
import pickle
import random
import sys

directory = os.path.join(os.getcwd(), '../../../../Service\service\eeg')
# print(os.path.abspath(directory))

sys.path.append(directory)
from eeg_data_check import Filter, FFTDataCheck

#### 读取学生数据，排除掉中重度抑郁的学生

In [4]:
# 读取 Excel 文件
file_path = '../../../Data/healthy_scale_results.xlsx'  # 将此替换为您的 Excel 文件路径
df = pd.read_excel(file_path, sheet_name='SDS')

valid_cols = df.columns.values[3::]

df['score'] = df[valid_cols].sum(axis=1) / 80 * 100
# print(df[['ID', 'score']])

invalid_negative_subjects_SDS = df[df['score'] >= 60]['ID'].tolist()
print("SDS invalid subjects: ", invalid_negative_subjects_SDS)

valid_negative_subjects_SDS = df[df['score'] < 60]['ID'].tolist()
print("SDS valid subjects: ", valid_negative_subjects_SDS)

SDS invalid subjects:  ['A04', 'A07', 'A09', 'A11', 'A15', 'A21', 'A25', 'A27', 'A29', 'A32', 'A33', 'A39', 'A48', 'A52']
SDS valid subjects:  ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A12', 'A13', 'A14', 'A16', 'A17', 'A18', 'A19', 'A20', 'A22', 'A23', 'A24', 'A26', 'A28', 'A30', 'A34', 'A35', 'A36', 'A37', 'A38', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A49', 'A50', 'A51', 'A53', 'A54', 'A55']


In [3]:
sample_seconds = 0.9*1  # Each sample window seconds
sample_freq = 250 # 250hz 

sample_length = int(sample_freq * sample_seconds)


print(f"sample length: {sample_length}")

sample length: 225


In [4]:
# positive_subject_dir = r"C:\Users\Administrator\Desktop\PPG_EEG_dataset\unhealthy"
# negative_subject_dir = r"C:\Users\Administrator\Desktop\PPG_EEG_dataset\healthy"

positive_subject_dir = r"C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\unhealthy"
negative_subject_dir = r"C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy"


In [5]:
invalid_negative_subjects = ['A41', 'A42', 'A46'] + invalid_negative_subjects_SDS
print(invalid_negative_subjects)

['A41', 'A42', 'A46', 'A04', 'A07', 'A09', 'A11', 'A15', 'A21', 'A25', 'A27', 'A29', 'A32', 'A33', 'A39', 'A48', 'A52']


In [6]:

def get_negative_subjects(directory):
    try:
        # 获取文件夹下所有文件和文件夹的名字
        all_files = os.listdir(directory)
        # 过滤出以 'eeg' 开头的文件
        eeg_files = [file for file in all_files if file.endswith('.json')]
        valid_subjects = []
        seen = set()
        for file in eeg_files:
            subject = file.split('_')[0]
            if subject not in seen and subject not in invalid_negative_subjects:
                valid_subjects.append(subject)
            seen.add(subject)
        return valid_subjects
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# 使用示例
negative_subjects = get_negative_subjects(negative_subject_dir)
print(negative_subjects)

['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A12', 'A13', 'A14', 'A16', 'A17', 'A18', 'A19', 'A20', 'A22', 'A23', 'A24', 'A26', 'A28', 'A30', 'A31', 'A34', 'A35', 'A36', 'A37', 'A38', 'A40', 'A43', 'A44', 'A45', 'A47', 'A49', 'A50', 'A51', 'A53', 'A54', 'A55']


In [7]:
print("negative subject count: ", len(negative_subjects))

negative subject count:  38


In [8]:
invalid_postive_subjects = ['2A27', '2A33']

In [7]:
label_filepath = r'../../../Data/unhealthy_label.txt'

In [9]:

def get_positive_subjects(lable_filepath):
    userids = []
    try:
        with open(lable_filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(',')
                if len(parts) == 3 and parts[1] == '1' and parts[0] not in invalid_postive_subjects:
                    userids.append(parts[0])
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return userids

positive_subjects = get_positive_subjects(label_filepath)
print("positive subject counts: ", len(positive_subjects))

positive subject counts:  30


In [11]:
random.shuffle(positive_subjects)
random.shuffle(negative_subjects)

print(f"positive subjects count: {len(positive_subjects)}", positive_subjects)
print(f"negative subjects count: {len(negative_subjects)}", negative_subjects)

positive subjects count: 30 ['20240619_2A02', '20240710_2A37', '20240710_2A09', '20240710_2A23', '20240619_2A33', '20240717_2A17', '20240619_2A50', '20240710_2A06', '20240717_2A20', '20240619_2A37', '20240619_2A09', '20240717_2A21', '20240619_2A27', '20240619_2A07', '20240710_2A30', '20240717_2A12', '20240710_2A05', '20240619_2A21', '20240710_2A28', '20240619_2A22', '20240717_2A51', '20240703_2A15', '20240626_2A50', '20240619_2A15', '20240710_2A16', '20240710_2A31', '20240703_2A50', '20240626_2A13', '20240626_2A26', '20240619_2A19']
negative subjects count: 38 ['A18', 'A44', 'A17', 'A45', 'A13', 'A26', 'A35', 'A12', 'A53', 'A54', 'A55', 'A08', 'A05', 'A23', 'A49', 'A47', 'A01', 'A03', 'A31', 'A37', 'A43', 'A22', 'A20', 'A34', 'A38', 'A50', 'A40', 'A36', 'A02', 'A06', 'A30', 'A24', 'A10', 'A19', 'A28', 'A51', 'A16', 'A14']


In [12]:
test_count = 8

In [13]:
# train_positive_subjects = positive_subjects[0: len(positive_subjects)-test_count]
# test_positive_subjects = positive_subjects[len(positive_subjects)-test_count::]

# train_negative_subjects = negative_subjects[0:len(negative_subjects)-test_count]
# test_negative_subjects = negative_subjects[len(negative_subjects)-test_count::]

# print("train positive subjects: ", train_positive_subjects)
# print("test positive subjects: ", test_positive_subjects)

# print("train negative subjects: ", train_negative_subjects)
# print("test negative subjects: ", test_negative_subjects)

In [14]:
# train_positive_subjects = positive_subjects[0: len(positive_subjects)-test_count]
# test_positive_subjects = positive_subjects[len(positive_subjects)-test_count::]

# test_negative_subjects_collection = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30']
# random.shuffle(test_negative_subjects_collection)

# test_negative_subjects = test_negative_subjects_collection[0:test_count]
# train_negative_subjects = [subject for subject in negative_subjects if subject not in test_negative_subjects]

# print("train positive subjects: ", train_positive_subjects)
# print("test positive subjects: ", test_positive_subjects)

# print("train negative subjects: ", train_negative_subjects)
# print("test negative subjects: ", test_negative_subjects)

In [15]:
test_positive_subjects = ['20240626_2A26', '20240619_2A15', '20240619_2A50', '20240710_2A30', '20240626_2A13', '20240710_2A28', '20240710_2A05', '20240710_2A31', '20240717_2A51']
train_positive_subjects = [subject for subject in positive_subjects if subject not in test_positive_subjects]

# test_negative_subjects = ['A05', 'A07', 'A13', 'A18', 'A20', 'A21', 'A24', 'A26', 'A31']

test_negative_subjects_collection = valid_negative_subjects_SDS[0:21]
random.shuffle(test_negative_subjects_collection)

test_negative_subjects = test_negative_subjects_collection[0:len(test_positive_subjects)]
train_negative_subjects = [subject for subject in negative_subjects if subject not in test_negative_subjects]

print("train positive subjects: ", train_positive_subjects)
print("test positive subjects: ", test_positive_subjects)

print("train negative subjects: ", train_negative_subjects)
print("test negative subjects: ", test_negative_subjects)


train positive subjects:  ['20240619_2A02', '20240710_2A37', '20240710_2A09', '20240710_2A23', '20240619_2A33', '20240717_2A17', '20240710_2A06', '20240717_2A20', '20240619_2A37', '20240619_2A09', '20240717_2A21', '20240619_2A27', '20240619_2A07', '20240619_2A21', '20240619_2A22', '20240717_2A51', '20240703_2A15', '20240626_2A50', '20240710_2A16', '20240703_2A50', '20240619_2A19']
test positive subjects:  ['20240626_2A26', '20240619_2A15', '20240619_2A50', '20240710_2A30', '20240626_2A13', '20240710_2A28', '20240710_2A05', '20240710_2A31', '20240717_2A12']
train negative subjects:  ['A18', 'A44', 'A45', 'A13', 'A26', 'A35', 'A12', 'A53', 'A54', 'A55', 'A05', 'A23', 'A49', 'A47', 'A01', 'A03', 'A31', 'A37', 'A43', 'A20', 'A34', 'A38', 'A50', 'A40', 'A36', 'A19', 'A51', 'A16', 'A14']
test negative subjects:  ['A17', 'A22', 'A24', 'A30', 'A10', 'A06', 'A08', 'A28', 'A02']


In [16]:

data_filter = FFTDataCheck()

In [17]:
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            # print("reading json file: ", file_path)
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def _get_samples(base_dir: str, target_subjects: list, label: int, data_aug: float=1.0):
    samples_ret = []
    for subject in target_subjects:
        files = os.listdir(base_dir)
        for file in files:
            if file.startswith(subject) and file.endswith('.json'):
                file_path = os.path.join(base_dir, file)
                json_data = read_json_file(file_path)
                left_wave_all, right_wave_all = [], []
                for raw_data in json_data['ProcessedDatas']:
                    left_wave = raw_data['LeftWave']
                    right_wave = raw_data['RightWave']

                    left_wave_all += left_wave
                    right_wave_all += right_wave


                if(len(left_wave_all) != len(right_wave_all)):
                    raise("error!!! left wave count not equal right wave count")
                
                samples = []
                step = int(sample_length / data_aug)

                # print(f"step: {step}")
                for idx in range(0, len(left_wave_all), step):
                        # print("idx: ", idx)
                        sub_left_wave = left_wave_all[idx:idx+sample_length]
                        sub_right_wave = right_wave_all[idx:idx+sample_length]
                        if len(sub_left_wave) != sample_length or len(sub_right_wave) != sample_length:
                            continue

                        left_valid = data_filter.is_valid(sub_left_wave)
                        right_valid = data_filter.is_valid(sub_right_wave)

                        # print("left_valid: ", left_valid, " right_valid: ", right_valid)
                        if left_valid and right_valid:
                            left_fft = np.abs(np.fft.fft(sub_left_wave))[:64]
                            left_fft = left_fft / np.sum(left_fft)

                            right_fft = np.abs(np.fft.fft(sub_right_wave))[:64]
                            right_fft = right_fft / np.sum(right_fft)

                            feat = np.concatenate([left_fft, right_fft], axis=0)
                            # print(len(feat))
                            samples.append((subject, feat, label))
                samples_ret += samples
                print(f"read file: {file_path} left_wave_all: {len(left_wave_all)} sample length: {sample_length} data aug: {data_aug} step: {step}, samples count: {len(samples)}")
                
    return samples_ret


def get_train_negative_samples():
    return _get_samples(base_dir=negative_subject_dir, target_subjects=train_negative_subjects, label=0, data_aug=2)

def get_test_negative_samples():
    return _get_samples(base_dir=negative_subject_dir, target_subjects=test_negative_subjects, label=0)


def get_train_positive_samples():
    return _get_samples(base_dir=positive_subject_dir, target_subjects=train_positive_subjects, label=1, data_aug=8)

def get_test_positive_samples():
    return _get_samples(base_dir=positive_subject_dir, target_subjects=test_positive_subjects, label=1)


train_negative_samples = get_train_negative_samples()
test_negative_samples = get_test_negative_samples()

train_positive_samples = get_train_positive_samples()
test_positive_samples = get_test_positive_samples()


read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A18_EEG.json left_wave_all: 143550 sample length: 225 data aug: 2 step: 112, samples count: 1264
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A44_1.json left_wave_all: 168750 sample length: 225 data aug: 2 step: 112, samples count: 1496
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A44_2.json left_wave_all: 141300 sample length: 225 data aug: 2 step: 112, samples count: 1253
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A44_3.json left_wave_all: 142650 sample length: 225 data aug: 2 step: 112, samples count: 1263
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A45_1.json left_wave_all: 155250 sample length: 225 data aug: 2 step: 112, samples count: 1323
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A45_2.json left_wave_all: 88650 sample length: 22

In [18]:
print(f"len(train_negative_samples): {len(train_negative_samples)}")

len(train_negative_samples): 100007


In [19]:
print("train_negative_samples count: ", len(train_negative_samples))
print("train_positive_samples count: ", len(train_positive_samples))

print("test_negative_samples count: ", len(test_negative_samples))
print("test_positive_samples count: ", len(test_positive_samples))


MIN = min(len(test_negative_samples), len(test_positive_samples))

random.shuffle(test_negative_samples)
random.shuffle(test_positive_samples)

# test_negative_samples = test_negative_samples[0:MIN]
# test_positive_samples = test_positive_samples[0:MIN]

print("\nAfter truncate: ")
print("test_negative_samples count: ", len(test_negative_samples))
print("test_positive_samples count: ", len(test_positive_samples))


train_negative_samples count:  100007
train_positive_samples count:  103262
test_negative_samples count:  6668
test_positive_samples count:  6139

After truncate: 
test_negative_samples count:  6668
test_positive_samples count:  6139


In [20]:
random.shuffle(train_negative_samples)
random.shuffle(train_positive_samples)
random.shuffle(test_negative_samples)
random.shuffle(test_positive_samples)

In [21]:
sample_folder = './samples'
if not os.path.exists(sample_folder):
    os.makedirs(sample_folder)

with open(os.path.join(sample_folder, 'train_negative_samples.pkl'), 'wb') as f:
    pickle.dump(train_negative_samples, f)

with open(os.path.join(sample_folder, 'train_positive_samples.pkl'), 'wb') as f:
    pickle.dump(train_positive_samples, f)

with open(os.path.join(sample_folder, 'test_negative_samples.pkl'), 'wb') as f:
    pickle.dump(test_negative_samples, f)

with open(os.path.join(sample_folder, 'test_positive_samples.pkl'), 'wb') as f:
    pickle.dump(test_positive_samples, f)
 

In [22]:
print(len(train_negative_samples[0][1]))
print(len(train_positive_samples[0][1]))
print(len(test_negative_samples[0][1]))
print(len(test_positive_samples[0][1]))

128
128
128
128
